Abhängigkeiten installieren und Packages laden

In [ ]:
!pip install pandas
!pip install numpy

import pandas as pd
import numpy as np

# wenn es nicht läuft runtime neu starten

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
  A

Subsets zusammenführen, Dopplungen entfernen

In [ ]:
csv_data_raw = [r'/content/4_1_ergebnisse.csv', r'/content/4_2_ergebnisse.csv', r'/content/4_3_ergebnisse.csv', r'/content/4_4_ergebnisse.csv']

csv_pfad = '/content/05_csv_data_raw.csv' # nicht wundern, hatte datei zwischenzeitlich umbenannt

csv_no_doubles_pfad = '/content/06_data_raw_no_doubles.csv'

df = pd.concat((pd.read_csv(f) for f in csv_data_raw), ignore_index=True)

df.to_csv('csv_data_raw.csv', index=False)

def entferne_doppelte_rezensionen(csv_pfad, anzahl_rezensionen=5):
    df = pd.read_csv(csv_pfad)

    # Doppelte Rezensionen basierend auf place_id und rezension entfernen
    df = df.drop_duplicates(subset=['place_id', 'rezension'], keep='first')

    # Anzahl der Rezensionen pro Ort begrenzen
    df = df.groupby('place_id').head(anzahl_rezensionen).reset_index(drop=True)

    return df

# Funktionsaufruf
ergebnis_df = entferne_doppelte_rezensionen(csv_pfad)

# Das bereinigte DataFrame anzeigen (optional)
print(ergebnis_df)

# Das bereinigte DataFrame in eine neue CSV-Datei speichern (optional)
ergebnis_df.to_csv("06_data_raw_no_doubles.csv", index=False)

Validierung der automatischen Abrufe: Zufallsstichprobe & manueller Abgleich

In [ ]:
df = pd.read_csv(csv_no_doubles_pfad) #siehe oben

def zufallsstichprobe(csv_pfad, stichprobengroesse=100, zufalls_seed=42): #seed 42
    df = pd.read_csv(csv_no_doubles_pfad)

    # Zufallsstichprobe ziehen
    stichprobe_df = df.sample(n=min(stichprobengroesse, len(df)), random_state=zufalls_seed)

    return stichprobe_df

# Beispielaufruf:
csv_datei = csv_no_doubles_pfad # Ersetzen Sie dies durch den tatsächlichen Dateinamen
stichprobe = zufallsstichprobe(csv_datei, stichprobengroesse=100, zufalls_seed=42)

# Die Stichprobe in eine neue CSV-Datei speichern (optional):
stichprobe.to_csv("07_stichprobe_validierung.csv", index=False)

# Validierung manuell durchgeführt, für Kriterien siehe Hausarbeit;
# Kodierung: 1 = valid; 0 = not valid or valid with restrictions;


########## hier noch die Ergebnisse ausrechnen ###############

                                                     ort  \
2507   Alexius/Josef Krankenhaus, Montanusstraße 54, ...   
11482  Sophien- und Hufeland-Klinikum gGmbH, Henry-va...   
4207   Ev. Krankenhaus Lippstadt, Wiedenbrücker Straß...   
1935   Rotes-Kreuz-Krankenhaus Bremen gGmbH, St.-Paul...   
7768   Klinik Dr. Wilhelm, Hans-Urmiller-Ring 46, Wol...   
...                                                  ...   
10769  Hauptstandort Rudolf-Virchow-Klinikum, Virchow...   
10563  Sana HANSE-Klinikum Wismar GmbH, Dr.-Unruh-Str...   
8830   Bezirksklinikum Ansbach, Paracelsusstraße 36, ...   
5006   Helios Kliniken Kassel, Hansteinstraße 29, Kassel   
4216       LWL-Klinik Lippstadt, Im Hofholz 6, Lippstadt   

                                        name                    strasze  \
2507               Alexius/Josef Krankenhaus             Montanusstraße   
11482   Sophien- und Hufeland-Klinikum gGmbH  Henry-van-de-Velde-Straße   
4207               Ev. Krankenhaus Lippstadt       Wie

initialize BERTopic

In [ ]:
!pip install bertopic
!pip install umap
from bertopic import BERTopic
from umap import UMAP

rezensionen_data = pd.read_csv("/content/06_data_raw_no_doubles.csv", header = None)

rezensionen_data['ort'] = rezensionen_data[0]
rezensionen_data['bundesland'] = rezensionen_data[5]
rezensionen_data['place_id'] = rezensionen_data[6]
rezensionen_data['bewertung'] = rezensionen_data[7]
rezensionen_data['rezension'] = rezensionen_data[8]
rezensionen_data['zeit'] = rezensionen_data[9]

rezensionen_data['bundesland'].value_counts()
rezensionen_data['zeit'].value_counts()
rezensionen_data['bewertung'].value_counts()

print(rezensionen_data.iloc[0:3, 3:6]) # Note that Python starts with 0 and excludes last element

NameError: name 'X' is not defined

run BERT

In [ ]:
umap_model = UMAP(random_state=42)

topic_model = BERTopic(umap_model=umap_model) # Use `BERTopic(language="multilingual")` to select a model that supports 50+ languages.
topics, probs = topic_model.fit_transform(rezensionen_data['rezension'])

topic_model.get_topic_info() # topics; -1 = garbage topic

topic_model.get_document_info(rezensionen_data['rezension']) # Klassifizierung der einzelnen Rezensionen

visualize BERT

In [ ]:
# topic distance
topic_model.visualize_topics() # topics are closer together if they exhibit similar word distributions or co-occurring patterns, even if they do not share identical words


In [ ]:
# word scores
topic_model.visualize_barchart()

# Plot for topics 40 to 51
topic_model.visualize_barchart(topics=list(range(40, 52)))

In [ ]:
# Topic similarity
topic_model.visualize_heatmap()

In [ ]:
# Topic hierarchy
topic_model.visualize_hierarchy()

deepdive into topics

In [ ]:
topic_model.get_topic(49) # Wörter anzeigen lassen

topic_model.get_representative_docs(topic=82) # Repräsentative Artikel anzeigen lassen

topic_model.set_topic_labels({49: "Iraq attacks", 82: "India attacks"}) # manuelles labeling
topic_model.get_topic_info(49) # default label = XX_top_three_words
topic_model.get_topic_info(82)

Topics je nach Covariate

In [ ]:
sterne = rezensionen_data['bewertung']

topics_x_sterne = topic_model.topics_per_class(rezensionen_data['description'], classes = sterne)

topic_model.visualize_topics_x_sterne(topics_x_sterne, top_n_topics = 31)

Advanced ...